In [1]:
import matplotlib
import sys
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import time

start_time = time.time()

#Ask for spin array length
N=int(input("Please, set the length N of the NxN spin array:"))
#Number of Montecarlo sweeps for equilibration
eqSteps = 100       
# Number of Montecarlo sweeps for calculation
mcSteps = 10000      
# Number of measurements
mcMeasure=mcSteps/10
T = np.arange(1, 3.1, 0.1);
C,E2 = np.zeros(len(T)), np.zeros(len(T))
n = 1/(N*N)

#Generate initial state in which spins in upper half array point in one direction, and the lower half in the opposite. For Kawasaki this will analogous to the equilibrium state at low temperature, which is the starting regime
def initialstate(N):

    initspin=np.zeros((N,N), dtype=float)
    
    if (N % 2) == 0:
        for i in range(N):
            for j in range(N):
                if i < (int(N/2)):
                    initspin[i,j]=1
                elif i > (int(N/2)):
                    initspin[i,j]=-1
                else:
                    initspin[i,j]=-1       
    else:
        for i in range(N):
            for j in range(N):
                if i < (int(N/2)):
                    initspin[i,j]=1
                elif i > (int(N/2)):
                    initspin[i,j]=-1
                else:
                    if j < (int(N/2)):
                        initspin[i,j]=-1    
                    else:
                        initspin[i,j]=1

    return initspin

#Calculate energy of a given spin configuration
def calcEnergy(spin):
    
    Totalenergy=0
    for i in range(N):
        for j in range(N):
            if i == (N-1):
                if j == (N-1) :
                    energy=spin[i,j]*(spin[0,j]+spin[i,0]+spin[i-1,j]+spin[i,j-1])

                else:

                    if j == 0:
                        energy=-spin[i,0]*(spin[0,0]+spin[i,1]+spin[i-1,0]+spin[i,N-1])

                    else:
                        energy=-spin[i,j]*(spin[0,j]+spin[i,j+1]+spin[i-1,j]+spin[i,j-1])
            elif i==0:

                if j == (N-1):
                    energy=-spin[0,j]*(spin[0,0]+spin[1,j]+spin[N-1,j]+spin[0,j-1])

                elif j == 0:
                    energy=-spin[0,0]*(spin[0,1]+spin[1,0]+spin[N-1,0]+spin[0,N-1])

                else:
                    energy=-spin[0,j]*(spin[0,j+1]+spin[1,j]+spin[N-1,j]+spin[0,j-1])

            else:

                if j == (N-1) :
                    energy=-spin[i,j]*(spin[i+1,j]+spin[i,0]+spin[i-1,j]+spin[i,j-1])

                elif j == 0:
                    energy=-spin[i,0]*(spin[i+1,0]+spin[i,1]+spin[i-1,0]+spin[i,N-1])

                else:
                    energy=-spin[i,j]*(spin[i+1,j]+spin[i,j+1]+spin[i-1,j]+spin[i,j-1])

            Totalenergy+=energy
    
    return Totalenergy/2

#Monte carlo using Kawasaki´s algorithm
def moveKawasaki(spin,beta):
    
    for m in range(N):
        for n in range(N):
            
            while True:
            
                i = np.random.randint(0, N)
                j = np.random.randint(0, N)
                a = np.random.randint(0, N)
                b = np.random.randint(0, N)

                spin1 = spin[i,j]
                spin2 = spin[a,b]
            

                if np.sign(spin1) != np.sign(spin2):
                
                    break
                
            if i == (N-1):
                if j == (N-1) :
                    sumspins1 = spin[0,j]+spin[i,0]+spin[i-1,j]+spin[i,j-1]
        
                else:
            
                    if j == 0:
                        sumspins1 = spin[0,0]+spin[i,1]+spin[i-1,0]+spin[i,N-1]
        
                    else:
                        sumspins1 = spin[0,j]+spin[i,j+1]+spin[i-1,j]+spin[i,j-1]
                    
            elif i==0:

                if j == (N-1):
                    sumspins1 = spin[0,0]+spin[1,j]+spin[N-1,j]+spin[0,j-1]
        
                elif j == 0:
                    sumspins1 = spin[0,1]+spin[1,0]+spin[N-1,0]+spin[0,N-1]
        
                else:
                    sumspins1 = spin[0,j+1]+spin[1,j]+spin[N-1,j]+spin[0,j-1]
    
            else:
        
                if j == (N-1) :
                    sumspins1 = spin[i+1,j]+spin[i,0]+spin[i-1,j]+spin[i,j-1]
        
                elif j == 0:
                    sumspins1 = spin[i+1,0]+spin[i,1]+spin[i-1,0]+spin[i,N-1]
        
                else:
                    sumspins1 = spin[i+1,j]+spin[i,j+1]+spin[i-1,j]+spin[i,j-1]
                    
                    
                    
            if a == (N-1):
                        
                if b == (N-1) :
                    sumspins2 = spin[0,b]+spin[a,0]+spin[a-1,b]+spin[a,b-1]
        
                else:
            
                    if b == 0:
                        sumspins2 = spin[0,0]+spin[a,1]+spin[a-1,0]+spin[a,N-1]
        
                    else:
                        sumspins2 = spin[0,b]+spin[a,b+1]+spin[a-1,b]+spin[a,b-1]
                    
            elif a==0:

                if b == (N-1):
                    sumspins2 = spin[0,0]+spin[1,b]+spin[N-1,b]+spin[0,b-1]
        
                elif b == 0:
                    sumspins2 = spin[0,1]+spin[1,0]+spin[N-1,0]+spin[0,N-1]
        
                else:
                    sumspins2 = spin[0,b+1]+spin[1,b]+spin[N-1,b]+spin[0,b-1]
    
            else:
        
                if b == (N-1) :
                    sumspins2 = spin[a+1,b]+spin[a,0]+spin[a-1,b]+spin[a,b-1]
        
                elif b == 0:
                    sumspins2 = spin[a+1,0]+spin[a,1]+spin[a-1,0]+spin[a,N-1]
        
                else:
                    sumspins2 = spin[a+1,b]+spin[a,b+1]+spin[a-1,b]+spin[a,b-1]
            
            Modulus = abs(i-a) + abs(b-j)
            
            if Modulus == 1:
                Ecost=((sumspins1-sumspins2)*(2*spin1))+4
            elif Modulus == (N-1):
                Ecost=((sumspins1-sumspins2)*(2*spin1))+4
            else:
                Ecost=((sumspins1-sumspins2)*(2*spin1))
                
            #if i == a and abs(b-j) == 1:
            #    Ecost=((sumspins1-sumspins2)*(2*spin1))+4
            #    
            #elif j == b and abs(i-a) == 1:
            #    Ecost=((sumspins1-sumspins2)*(2*spin1))+4
            #else:
            #    Ecost=((sumspins1-sumspins2)*(2*spin1))
                
            p=min(1, np.exp(-Ecost*beta))
            randomnum=np.random.rand()
                
            if randomnum < p :
                spin1*=-1
                spin2*=-1                
            spin[i,j] = spin1
            spin[a,b] = spin2
    
    
    return spin


Ene=np.zeros((len(T),int(mcMeasure)))
Mag=np.zeros((len(T),int(mcMeasure)))

#Set first spin configuration using initial state function
spin = initialstate(N)

for t in range(len(T)):
    
    beta=1.0/T[t]
    
    #Equilibrate system
    for i in range(eqSteps):         
        moveKawasaki(spin, beta)

    #Run system and take energy measurements
    for i in range(int(mcMeasure)):
        for j in range(10):
            moveKawasaki(spin, beta)
        Ene[t][i] = calcEnergy(spin)    
    
#Create file for energy data       
fenergyKawasaki=pd.DataFrame(Ene)

fenergyKawasaki.to_csv('fenergyKawasakiN'+str(N)+'.csv',index=False)
        
#Open energy data file 
EnergyDat = pd.read_csv("fenergyKawasakiN"+str(N)+".csv")

#Compute energy mean from data file
Eplot= EnergyDat.mean(axis=1)
#Compute squared energy mean from data file
E2=np.average(EnergyDat**2,axis=1)
#Calculate heat capacity
C = (E2 - (Eplot**2) )/((T**2)*(N**2))
#Create heat capacity data file
fCKawasaki=pd.DataFrame(C)
fCKawasaki.to_csv('fCKawasakiN'+str(N)+'.csv', index = False)

 
print("It took: "+str((time.time()-start_time)/60)+" minutes to run")

Please, set the length N of the NxN spin array:10
It took: 12.991047286987305 minutes to run
